# Additional real dataset results:

In [5]:
import numpy as np
import pandas as pd
import os

from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-white")

from clover.locart import LocalRegressionSplit, LocartSplit, MondrianRegressionSplit, RegressionSplit
from acpi import ACPI
from clover.scores import RegressionScore, LocalRegressionScore
import time

from clover.locart import LocartSplit, MondrianRegressionSplit
from clover.scores import RegressionScore
from clover.utils import compute_interval_length, split, smis
import gc

original_path = os.getcwd()
# one folder back
folder_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# figure path
images_dir = "figures"

In [6]:
original_path

'/home/kuben45/estatistica_UFSCar/Doutorado/clover/examples'

## Defining function to run big dataset experiment:

In [7]:
def obtain_main_metrics_real(
    data_name,
    is_fitted = True,
    figname_tree = "tree",
    base_model = RandomForestRegressor,
    sig = 0.1, 
    test_size = 0.2,
    calib_size = 0.5, 
    random_seed = 1250,
    split_calib = False,
    plot_tree = False,
    nbins = 30,
    criterion = "squared_error",
    max_depth = None,
    max_leaf_nodes = None,
    min_samples_leaf = 150,
    prune = True,
    random_projections=False,
    h=20,
    m=300,
    n_estimators = 100,
    **kwargs):

    # importing data, selecting some rows and then splitting
    data_path = os.path.normpath(os.getcwd() + os.sep + os.pardir) + "/data/processed/" + data_name + ".csv"

    # reading data using pandas
    data = pd.read_csv(data_path)
    y = data["target"].to_numpy()
    X = (data.
    drop("target", axis = 1).
    to_numpy())

    print("Number of samples used for training and calibration: {}".format((1 - test_size)*X.shape[0]))
    print("Number of samples used for testing: {}".format(test_size*X.shape[0]))

    # splitting data into train, calibration and test
    data = split(X, y, test_size, calib_size, calibrate = True, random_seed = random_seed)

    if data_name == "amazon":
        print("Encoding text data")
        X_train = data["X_train"].flatten()
        X_test = data["X_test"].flatten()
        X_calib = data["X_calib"].flatten()

        tfidf = TfidfVectorizer(max_features=500)
        X_train = tfidf.fit_transform(X_train).toarray()
        X_test = tfidf.transform(X_test).toarray()
        X_calib = tfidf.transform(X_calib).toarray()
        features = tfidf.get_feature_names_out()
        np.savetxt(folder_path + "/data/processed/amazon_features_test", features, fmt="%s")

        data["X_train"] = X_train
        data["X_test"] = X_test
        data["X_calib"] = X_calib

    # setting seed
    np.random.seed(random_seed)
    print("Fitting base model")
    model = base_model(**kwargs).fit(data["X_train"], data["y_train"])

    # fitting mondrian regression split
    print("Fitting mondrian regression split")
    start_mondrian_split = time.time()
    micp = MondrianRegressionSplit(model, is_fitted = is_fitted, alpha = sig, k = nbins, **kwargs)
    micp.fit(data["X_train"], data["y_train"])
    micp.calibrate(data["X_calib"], data["y_calib"], split = False)

    end_mondrian_split = time.time() - start_mondrian_split
    print("Time Elapsed to fit mondrian regression split: ", end_mondrian_split)

    print("Computing metrics")
    start_mondrian_split = time.time()
    # predictions for empirical metrics
    micp_pred = micp.predict(data["X_test"])

    # smis
    micp_smis = smis(micp_pred, data["y_test"], alpha = sig)
        

    # mean interval length
    micp_interval_len = np.mean(compute_interval_length(micp_pred))


    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= micp_pred[:, 0], 
        data["y_test"] <= micp_pred[:, 1]) + 0
    micp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    micp_interval_len_cover = np.mean(compute_interval_length(micp_pred[cover_idx]))


    end_mondrian_split = time.time() - start_mondrian_split
    print("Time Elapsed to compute statistics for mondrian regression split: ", end_mondrian_split)

    # deleting mondrian objects
    del micp
    del micp_pred
    del cover_idx
    del marg_cover
    gc.collect()

    print("Fitting locart")
    start_loc = time.time()
    # fitting locart
    locart_obj = LocartSplit(nc_score = RegressionScore, base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, **kwargs)
    locart_obj.fit(data["X_train"], data["y_train"])
    locart_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit Locart: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    locart_pred = np.array(locart_obj.predict(data["X_test"]))

    # smis
    locart_smis = smis(locart_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= locart_pred[:, 0], 
        data["y_test"] <= locart_pred[:, 1]) + 0
    locart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    locart_interval_len = np.mean(compute_interval_length(locart_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    locart_interval_len_cover = np.mean(compute_interval_length(locart_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for Locart: ", end_loc)

    # deleting locart objects
    del locart_obj
    del locart_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting normal difficulty locart
    print("Fitting A-locart to toy example:")
    start_loc = time.time()
    dlocart_obj = LocartSplit(nc_score = RegressionScore, cart_type = "CART", base_model = model, is_fitted = is_fitted,
                               alpha = sig, split_calib = split_calib, 
                              weighting = True, **kwargs)
    dlocart_obj.fit(data["X_train"], data["y_train"])
    dlocart_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit A-Locart: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    dlocart_pred = np.array(dlocart_obj.predict(data["X_test"]))

    # smis
    dlocart_smis = smis(dlocart_pred, data["y_test"], alpha = sig)

    # mean interval length
    dlocart_interval_len = np.mean(compute_interval_length(dlocart_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= dlocart_pred[:, 0], 
        data["y_test"] <= dlocart_pred[:, 1]) + 0
    dlocart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    dlocart_interval_len_cover = np.mean(compute_interval_length(dlocart_pred[cover_idx]))

    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for A-Locart: ", end_loc)
    # deleting alocart objects
    del dlocart_obj
    del dlocart_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting normal loforest 
    print("Fitting loforest")
    start_loc = time.time()
    # fitting loforest
    loforest_obj = LocartSplit(nc_score = RegressionScore, cart_type = "RF", base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, **kwargs)
    loforest_obj.fit(data["X_train"], data["y_train"])
    loforest_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune, n_estimators = n_estimators)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit Loforest: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    loforest_pred = np.array(loforest_obj.predict(data["X_test"]))

    # smis
    loforest_smis = smis(loforest_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= loforest_pred[:, 0], 
        data["y_test"] <= loforest_pred[:, 1]) + 0
    loforest_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    loforest_interval_len = np.mean(compute_interval_length(loforest_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    loforest_interval_len_cover = np.mean(compute_interval_length(loforest_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for Loforest: ", end_loc)

    # deleting loforest objects
    del loforest_obj
    del loforest_pred
    del cover_idx
    del marg_cover
    gc.collect()

    print("Fitting A-Loforest")
    start_loc = time.time()
    # fitting locart
    aloforest_obj = LocartSplit(nc_score = RegressionScore, cart_type = "RF", base_model = model, is_fitted = is_fitted, 
                             alpha = sig, split_calib = split_calib, weighting = True, **kwargs)
    aloforest_obj.fit(data["X_train"], data["y_train"])
    aloforest_obj.calib(data["X_calib"], data["y_calib"], max_depth = max_depth, 
    max_leaf_nodes = max_leaf_nodes, min_samples_leaf = min_samples_leaf, criterion = criterion, prune_tree = prune, n_estimators = n_estimators)
    
    end_loc = time.time() - start_loc
    print("Time Elapsed to fit A-Loforest: ", end_loc)

    print("Computing metrics")
    start_loc = time.time()
    # predictions
    aloforest_pred = np.array(aloforest_obj.predict(data["X_test"]))

    # smis
    aloforest_smis = smis(aloforest_pred, data["y_test"], alpha = sig)

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= aloforest_pred[:, 0], 
        data["y_test"] <= aloforest_pred[:, 1]) + 0
    aloforest_ave_marginal_cov = np.mean(
        marg_cover
    )

    # mean interval length
    aloforest_interval_len = np.mean(compute_interval_length(aloforest_pred))

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    aloforest_interval_len_cover = np.mean(compute_interval_length(aloforest_pred[cover_idx]))
    end_loc = time.time() - start_loc
    print("Time Elapsed to compute metrics for A-Loforest: ", end_loc)

    # deleting a-loforest objects
    del aloforest_obj
    del aloforest_pred
    del cover_idx
    del marg_cover
    gc.collect()


    print("Fitting regression split")
    start_split = time.time()
    icp = RegressionSplit(base_model = model, alpha = sig, is_fitted = True)
    icp.fit(data["X_train"], data["y_train"])
    icp.calibrate(data["X_calib"], data["y_calib"])

    end_split = time.time() - start_split
    print("Time Elapsed to fit regression split: ", end_split)

    print("Computing metrics")
    start_split = time.time()
    # predictions
    icp_pred = icp.predict(data["X_test"])

    # smis
    icp_smis = smis(icp_pred, data["y_test"], alpha = sig)

    # ICP interval length
    icp_interval_len = np.mean(compute_interval_length(icp_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= icp_pred[:, 0], 
        data["y_test"] <= icp_pred[:, 1]) + 0
    icp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    icp_interval_len_cover = np.mean(compute_interval_length(icp_pred[cover_idx]))

    end_split = time.time() - start_split
    print("Time Elapsed to compute statistics for regression split: ", end_split)

    # deleting ICP objects
    del icp
    del icp_pred
    del cover_idx
    del marg_cover
    gc.collect()

    # fitting wighted regression split
    print("Fitting weighted regression split")
    start_weighted_split = time.time()
    wicp = LocalRegressionSplit(model, is_fitted = is_fitted, alpha = sig, **kwargs)
    wicp.fit(data["X_train"], data["y_train"])
    wicp.calibrate(data["X_calib"], data["y_calib"])

    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to fit weighted regression split: ", end_weighted_split)

    print("Computing metrics")
    start_weighted_split = time.time()
    # predictions
    wicp_pred = wicp.predict(data["X_test"])

    # smis
    wicp_smis = smis(wicp_pred, data["y_test"], alpha = sig)

    # ICP interval length
    wicp_interval_len = np.mean(compute_interval_length(wicp_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= wicp_pred[:, 0], 
        data["y_test"] <= wicp_pred[:, 1]) + 0
    wicp_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    wicp_interval_len_cover = np.mean(compute_interval_length(wicp_pred[cover_idx]))
    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to compute statistics for weighted regression split: ", end_weighted_split)

    # deleting WICP objects
    del wicp
    del wicp_pred
    del cover_idx
    del marg_cover
    gc.collect()


    start_loc = time.time()

    print("Fitting W-LOFOREST")
    wlocart_obj = LocartSplit(
        nc_score=LocalRegressionScore,
        cart_type="RF",
        base_model=model,
        is_fitted=is_fitted,
        alpha=sig,
        split_calib=split_calib,
        **kwargs,
    )
    wlocart_obj.fit(data["X_train"], data["y_train"])
    wlocart_obj.calib(
        data["X_calib"],
        data["y_calib"],
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        prune_tree=prune,
        random_projections=random_projections,
        n_estimators = n_estimators,
        m=m,
        h=h,
    )

    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to fit W-LOFOREST: ", end_weighted_split)

    print("Computing metrics")
    start_weighted_split = time.time()
    # predictions
    wlocart_pred = wlocart_obj.predict(data["X_test"])

    # smis
    wlocart_smis = smis(wlocart_pred, data["y_test"], alpha = sig)

    # ICP interval length
    wlocart_interval_len = np.mean(compute_interval_length(wlocart_pred))

    # marginal coverage
    marg_cover = np.logical_and(data["y_test"] >= wlocart_pred[:, 0], 
        data["y_test"] <= wlocart_pred[:, 1]) + 0
    wlocart_ave_marginal_cov = np.mean(
        marg_cover
    )

    # interval length | coveraqe
    cover_idx = np.where(marg_cover == 1)
    wlocart_interval_len_cover = np.mean(compute_interval_length(wlocart_pred[cover_idx]))
    end_weighted_split = time.time() - start_weighted_split
    print("Time Elapsed to compute statistics for W-LOFOREST: ", end_weighted_split)

    # deleting WICP objects
    del wlocart_obj
    del wlocart_pred
    del cover_idx
    del marg_cover
    gc.collect()

    if data != "yearprediction":
        print("Fitting ACPI")
        start_weighted_split = time.time()
        acpi = ACPI(model_cali = model, n_estimators = 100)
        acpi.fit(data["X_test"], data["y_test"], nonconformity_func = None)
        acpi.fit_calibration(data["X_calib"], data["y_calib"], quantile = 1 - sig, only_qrf = True)

        end_weighted_split = time.time() - start_weighted_split
        print("Time Elapsed to fit ACPI: ", end_weighted_split)

        print("Computing metrics")
        start_weighted_split = time.time()
        # predictions
        acpi_pred = np.stack((acpi.predict_pi(data["X_test"], method="qrf")), axis=-1)

        # smis
        acpi_smis = smis(acpi_pred, data["y_test"], alpha = sig)

        # ICP interval length
        acpi_interval_len = np.mean(compute_interval_length(acpi_pred))

        # marginal coverage
        marg_cover = np.logical_and(data["y_test"] >= acpi_pred[:, 0], 
            data["y_test"] <= acpi_pred[:, 1]) + 0
        acpi_ave_marginal_cov = np.mean(
            marg_cover
        )

        # interval length | coveraqe
        cover_idx = np.where(marg_cover == 1)
        acpi_interval_len_cover = np.mean(compute_interval_length(acpi_pred[cover_idx]))
        end_weighted_split = time.time() - start_weighted_split
        print("Time Elapsed to compute statistics for ACPI: ", end_weighted_split)

        # deleting ACPI objects
        del acpi
        del acpi_pred
        del cover_idx
        del marg_cover
        gc.collect()
    
    
    
        all_results =  pd.DataFrame(data = {"Methods":["LOCART", "A-LOCART", "LOFOREST", "A-LOFOREST",
                                                    "Regresion split", "Weighted regression split", 
                                                    "Mondrian regression split", "ACPI", "W-LOFOREST"],
        "Average marginal coverage":[locart_ave_marginal_cov, dlocart_ave_marginal_cov, loforest_ave_marginal_cov, aloforest_ave_marginal_cov, 
                                    icp_ave_marginal_cov, wicp_ave_marginal_cov, micp_ave_marginal_cov, acpi_ave_marginal_cov,
                                    wlocart_ave_marginal_cov],
        "Average interval length":[locart_interval_len, dlocart_interval_len, loforest_interval_len, aloforest_interval_len,
                                icp_interval_len, wicp_interval_len, micp_interval_len, acpi_interval_len,
                                wlocart_interval_len],
        "Average interval length given coverage":[locart_interval_len_cover, dlocart_interval_len_cover, loforest_interval_len_cover, aloforest_interval_len_cover,
                                                icp_interval_len_cover, wicp_interval_len_cover, micp_interval_len_cover, acpi_interval_len_cover,
                                                wlocart_interval_len_cover],
        "SMIS values":[locart_smis, dlocart_smis, loforest_smis, aloforest_smis, icp_smis, wicp_smis, micp_smis, acpi_smis, wlocart_smis]})
    else:
        all_results =  pd.DataFrame(data = {"Methods":["LOCART", "A-LOCART", "LOFOREST", "A-LOFOREST",
                                                    "Regresion split", "Weighted regression split", 
                                                    "Mondrian regression split", "W-LOFOREST"],
        "Average marginal coverage":[locart_ave_marginal_cov, dlocart_ave_marginal_cov, loforest_ave_marginal_cov, aloforest_ave_marginal_cov, 
                                    icp_ave_marginal_cov, wicp_ave_marginal_cov, micp_ave_marginal_cov, wlocart_ave_marginal_cov],
        "Average interval length":[locart_interval_len, dlocart_interval_len, loforest_interval_len, aloforest_interval_len,
                                icp_interval_len, wicp_interval_len, micp_interval_len, wlocart_interval_len],
        "Average interval length given coverage":[locart_interval_len_cover, dlocart_interval_len_cover, loforest_interval_len_cover, aloforest_interval_len_cover,
                                                icp_interval_len_cover, wicp_interval_len_cover, micp_interval_len_cover, wlocart_interval_len_cover],
        "SMIS values":[locart_smis, dlocart_smis, loforest_smis, aloforest_smis, icp_smis, wicp_smis, micp_smis, wlocart_smis]})
        
    # plotting tree to visualize partitions
    if plot_tree:
        plt.figure(figsize=(25, 20))
        locart_obj.plot_locart()
        plt.savefig(f"{images_dir}/{figname_tree}")
        plt.show()
    
    return all_results

### Running wave energy (WEC) dataset ($n = 54006$, $p = 50$):

In [42]:
WEC_results = obtain_main_metrics_real(data_name = "WEC", random_seed = 1250, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 43205.600000000006
Number of samples used for testing: 10801.400000000001
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  1.1672310829162598
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  0.36299777030944824
Fitting locart
Time Elapsed to fit Locart:  7.889450788497925
Computing metrics
Time Elapsed to compute metrics for Locart:  0.14373016357421875
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  8.237366676330566
Computing metrics
Time Elapsed to compute metrics for A-Locart:  0.35291099548339844
Fitting loforest
Time Elapsed to fit Loforest:  20.60428476333618
Computing metrics
Time Elapsed to compute metrics for Loforest:  0.2610437870025635
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  24.923173427581787
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  0.4588911533355713
Fitting regression split
Time

100%|██████████| 50/50 [03:14<00:00,  3.89s/it]


Time Elapsed to fit ACPI:  202.34858870506287
Computing metrics
Time Elapsed to compute statistics for ACPI:  0.30381345748901367


In [43]:
WEC_results

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.904555,50844.513184,51140.551068,-92417.225005
1,A-LOCART,0.897241,42595.179480,43025.616519,-61211.005131
2,LOFOREST,0.929828,52097.710733,50388.995830,-90653.773705
3,A-LOFOREST,0.930476,42997.290654,41789.846765,-59532.489493
4,Regresion split,0.902425,67529.792502,67529.792502,-148460.137299
5,Weighted regression split,0.900389,32198.996192,28625.023694,-77295.689564
6,Mondrian regression split,0.929458,44539.309496,43287.143667,-66655.011907
7,ACPI,0.945195,45492.694340,42883.943486,-69075.870502
8,W-LOFOREST,0.923070,39986.333862,37816.181431,-69686.118129


Another seed to check stability:

In [44]:
WEC_results_v2 = obtain_main_metrics_real(data_name = "WEC", random_seed = 450, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 43205.600000000006
Number of samples used for testing: 10801.400000000001
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  1.1519865989685059
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  0.36353158950805664
Fitting locart
Time Elapsed to fit Locart:  6.76677393913269
Computing metrics
Time Elapsed to compute metrics for Locart:  0.13649892807006836
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  8.121527910232544
Computing metrics
Time Elapsed to compute metrics for A-Locart:  0.351534366607666
Fitting loforest
Time Elapsed to fit Loforest:  19.031859874725342
Computing metrics
Time Elapsed to compute metrics for Loforest:  0.25800085067749023
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  22.45592761039734
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  0.45488667488098145
Fitting regression split
Time 

100%|██████████| 50/50 [03:13<00:00,  3.87s/it]


Time Elapsed to fit ACPI:  201.5062370300293
Computing metrics
Time Elapsed to compute statistics for ACPI:  0.31244492530822754


In [45]:
WEC_results_v2

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.907887,54821.267779,54900.478742,-93794.340628
1,A-LOCART,0.903166,44315.157863,44620.743545,-62486.342422
2,LOFOREST,0.936864,53680.947839,51571.016296,-89383.752296
3,A-LOFOREST,0.922700,44228.880349,43628.890187,-60569.135055
4,Regresion split,0.900944,66255.726698,66255.726698,-149383.880518
5,Weighted regression split,0.899370,33537.047028,29955.875392,-76140.114956
6,Mondrian regression split,0.929920,46187.654894,44710.563282,-67774.084129
7,ACPI,0.942325,47248.760324,45162.387242,-68264.146272
8,W-LOFOREST,0.912701,37283.875322,34458.289423,-72259.780063


## Running amazon dataset ($n = 130000$, $p = 500$):

In [8]:
amazon_results = obtain_main_metrics_real(data_name = "amazon", random_seed = 0, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 104000.0
Number of samples used for testing: 26000.0
Encoding text data
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  12.188075065612793
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  3.4686660766601562
Fitting locart
Time Elapsed to fit Locart:  148.45488452911377
Computing metrics
Time Elapsed to compute metrics for Locart:  0.823732852935791
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  96.46421146392822
Computing metrics
Time Elapsed to compute metrics for A-Locart:  3.566587209701538
Fitting loforest
Time Elapsed to fit Loforest:  196.18423461914062
Computing metrics
Time Elapsed to compute metrics for Loforest:  1.510570764541626
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  119.83218121528625
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  4.090203285217285
Fitting regression split
Time Elapse

100%|██████████| 50/50 [08:30<00:00, 10.22s/it]


Time Elapsed to fit ACPI:  530.9527945518494
Computing metrics
Time Elapsed to compute statistics for ACPI:  1.372321367263794


In [9]:
amazon_results

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.894115,3.208903,3.216635,-4.874536
1,A-LOCART,0.897231,2.854969,2.852053,-4.674242
2,LOFOREST,0.901731,3.317693,3.308595,-4.854649
3,A-LOFOREST,0.900500,2.978737,2.969523,-4.601219
4,Regresion split,0.895692,3.740000,3.740000,-5.254985
5,Weighted regression split,0.896654,2.978706,3.018347,-4.783217
6,Mondrian regression split,0.898077,2.921707,2.916265,-4.672176
7,ACPI,0.902923,3.546778,3.525990,-4.960261
8,W-LOFOREST,0.905385,3.117135,3.158902,-4.743178


Another seed to check stability:

In [10]:
amazon_results_v2 = obtain_main_metrics_real(data_name = "amazon", random_seed = 250, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 104000.0
Number of samples used for testing: 26000.0
Encoding text data
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  8.302001476287842
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  2.6836085319519043
Fitting locart
Time Elapsed to fit Locart:  140.71266627311707
Computing metrics
Time Elapsed to compute metrics for Locart:  0.7882289886474609
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  79.31235790252686
Computing metrics
Time Elapsed to compute metrics for A-Locart:  2.407371759414673
Fitting loforest
Time Elapsed to fit Loforest:  155.28910374641418
Computing metrics
Time Elapsed to compute metrics for Loforest:  1.212578296661377
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  97.96655988693237
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  2.9519600868225098
Fitting regression split
Time Elapse

100%|██████████| 50/50 [08:44<00:00, 10.48s/it]


Time Elapsed to fit ACPI:  543.6025547981262
Computing metrics
Time Elapsed to compute statistics for ACPI:  1.498715877532959


In [11]:
amazon_results_v2

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.899385,3.261496,3.275513,-4.719060
1,A-LOCART,0.904846,2.845942,2.841785,-4.480990
2,LOFOREST,0.906885,3.334526,3.327056,-4.686064
3,A-LOFOREST,0.908500,2.994499,2.983817,-4.428139
4,Regresion split,0.901038,3.749000,3.749000,-5.116582
5,Weighted regression split,0.901038,3.004806,3.040392,-4.584949
6,Mondrian regression split,0.905077,2.960569,2.952463,-4.474659
7,ACPI,0.910308,3.550779,3.529771,-4.807285
8,W-LOFOREST,0.909500,3.122582,3.158577,-4.556146


## Running SGEMM dataset ($n = 241600$, $p = 14$)

In [8]:
SGEMM_results = obtain_main_metrics_real(data_name = "SGEMM", random_seed = 0, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 193280.0
Number of samples used for testing: 48320.0
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  5.99904990196228
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  1.9768235683441162
Fitting locart
Time Elapsed to fit Locart:  10.515614986419678
Computing metrics
Time Elapsed to compute metrics for Locart:  0.9096426963806152
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  23.66080379486084
Computing metrics
Time Elapsed to compute metrics for A-Locart:  1.9160857200622559
Fitting loforest
Time Elapsed to fit Loforest:  11.294396162033081
Computing metrics
Time Elapsed to compute metrics for Loforest:  1.5143074989318848
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  20.36793065071106
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  2.526036024093628
Fitting regression split
Time Elapsed to fit regression

100%|██████████| 50/50 [15:15<00:00, 18.30s/it]


Time Elapsed to fit ACPI:  954.290203332901
Computing metrics
Time Elapsed to compute statistics for ACPI:  1.4187836647033691


In [9]:
SGEMM_results

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.900290,13.153752,13.200687,-20.176214
1,A-LOCART,0.898075,11.276769,11.328973,-16.451613
2,LOFOREST,0.912748,12.986955,12.870984,-20.270092
3,A-LOFOREST,0.906705,11.266623,11.255110,-16.211601
4,Regresion split,0.899255,12.123055,12.123055,-30.476741
5,Weighted regression split,0.899110,12.198808,12.458120,-17.209902
6,Mondrian regression split,0.900310,11.426149,11.484364,-17.444256
7,ACPI,0.929118,13.027339,12.721901,-18.410295
8,W-LOFOREST,0.906478,12.640339,12.947779,-17.074079


Another seed to check stability:

In [10]:
SGEMM_results_v2 = obtain_main_metrics_real(data_name = "SGEMM", random_seed = 45, random_state = 45, min_samples_leaf = 150, n_estimators = 100)

Number of samples used for training and calibration: 193280.0
Number of samples used for testing: 48320.0
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  5.737327337265015
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  1.9587411880493164
Fitting locart
Time Elapsed to fit Locart:  11.542576313018799
Computing metrics
Time Elapsed to compute metrics for Locart:  0.9061675071716309
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  23.642030954360962
Computing metrics
Time Elapsed to compute metrics for A-Locart:  1.8942124843597412
Fitting loforest
Time Elapsed to fit Loforest:  11.314781665802002
Computing metrics
Time Elapsed to compute metrics for Loforest:  1.5072591304779053
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  19.749062538146973
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  2.504007577896118
Fitting regression split
Time Elapsed to fit regress

100%|██████████| 50/50 [15:17<00:00, 18.35s/it]


Time Elapsed to fit ACPI:  955.8356170654297
Computing metrics
Time Elapsed to compute statistics for ACPI:  1.4194200038909912


In [11]:
SGEMM_results_v2

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.900869,12.768699,12.735415,-20.514160
1,A-LOCART,0.900166,11.009699,10.961258,-16.918331
2,LOFOREST,0.911776,12.649414,12.466240,-20.586877
3,A-LOFOREST,0.907119,10.956778,10.828408,-16.818035
4,Regresion split,0.901014,12.015660,12.015660,-31.094719
5,Weighted regression split,0.899814,11.844832,12.010785,-17.353726
6,Mondrian regression split,0.899524,11.245128,11.187678,-18.113688
7,ACPI,0.928704,12.745174,12.340599,-18.527195
8,W-LOFOREST,0.903746,12.280472,12.512805,-17.154818


## Running year prediction dataset ($n = 463715$, $p = 90$):

In [12]:
year_prediction_results = obtain_main_metrics_real(data_name = "yearprediction", random_seed = 125, random_state = 45, min_samples_leaf = 1500, n_estimators = 100)

Number of samples used for training and calibration: 370972.0
Number of samples used for testing: 92743.0
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  16.268493175506592
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  5.009557247161865
Fitting locart
Time Elapsed to fit Locart:  217.27278900146484
Computing metrics
Time Elapsed to compute metrics for Locart:  2.1003975868225098
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  219.7062213420868
Computing metrics
Time Elapsed to compute metrics for A-Locart:  4.909548759460449
Fitting loforest
Time Elapsed to fit Loforest:  598.4454429149628
Computing metrics
Time Elapsed to compute metrics for Loforest:  3.045593023300171
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  613.9095451831818
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  5.882875442504883
Fitting regression split
Time Elapsed to fit regression s

100%|██████████| 50/50 [36:19<00:00, 43.59s/it]


Time Elapsed to fit ACPI:  2267.4374482631683
Computing metrics
Time Elapsed to compute statistics for ACPI:  4.3617753982543945


In [13]:
year_prediction_results

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.897362,26.561413,26.612974,-39.257427
1,A-LOCART,0.901081,24.729096,24.736335,-37.171050
2,LOFOREST,0.907130,26.612937,26.488184,-38.862721
3,A-LOFOREST,0.904241,24.827237,24.789130,-36.906230
4,Regresion split,0.900920,29.300000,29.300000,-43.630101
5,Weighted regression split,0.899421,25.082658,24.993910,-37.637042
6,Mondrian regression split,0.900984,24.734349,24.733714,-37.189868
7,ACPI,0.917439,27.960746,27.760560,-38.541479
8,W-LOFOREST,0.903152,25.501948,25.470054,-36.947225


Checking another seed:

In [14]:
year_prediction_results_v2 = obtain_main_metrics_real(data_name = "yearprediction", random_seed = 0, random_state = 45, min_samples_leaf = 1500, n_estimators = 100)

Number of samples used for training and calibration: 370972.0
Number of samples used for testing: 92743.0
Fitting base model
Fitting mondrian regression split
Time Elapsed to fit mondrian regression split:  15.219798803329468
Computing metrics
Time Elapsed to compute statistics for mondrian regression split:  4.970174074172974
Fitting locart
Time Elapsed to fit Locart:  203.31899881362915
Computing metrics
Time Elapsed to compute metrics for Locart:  2.0953123569488525
Fitting A-locart to toy example:
Time Elapsed to fit A-Locart:  219.8097050189972
Computing metrics
Time Elapsed to compute metrics for A-Locart:  4.847153663635254
Fitting loforest
Time Elapsed to fit Loforest:  605.3749315738678
Computing metrics
Time Elapsed to compute metrics for Loforest:  3.0329749584198
Fitting A-Loforest
Time Elapsed to fit A-Loforest:  621.5910792350769
Computing metrics
Time Elapsed to compute metrics for A-Loforest:  5.831326961517334
Fitting regression split
Time Elapsed to fit regression spl

100%|██████████| 50/50 [36:11<00:00, 43.42s/it]


Time Elapsed to fit ACPI:  2259.1092801094055
Computing metrics
Time Elapsed to compute statistics for ACPI:  4.438794136047363


In [15]:
year_prediction_results_v2

,Methods,Average marginal coverage,Average interval length,Average interval length given coverage,SMIS values
0,LOCART,0.899367,26.774725,26.832427,-39.519288
1,A-LOCART,0.898310,24.840953,24.886822,-37.360776
2,LOFOREST,0.907853,26.685222,26.560709,-39.079556
3,A-LOFOREST,0.901912,24.926168,24.914616,-37.119236
4,Regresion split,0.898591,29.100000,29.100000,-43.867730
5,Weighted regression split,0.899939,25.172579,25.076973,-37.904531
6,Mondrian regression split,0.899518,24.801688,24.814824,-37.432964
7,ACPI,0.915336,27.786557,27.574493,-38.824825
8,W-LOFOREST,0.902968,25.469715,25.409780,-37.243408


: 